<table>
<tr>
    <td width=10%>
        <img src="images/eisti_logo.png">
    </td>
    <td>
        <center>
            <h1>Deep Learning et Applications</h1>
        </center>
    </td>
    <td width=15%>
        Yann Vernaz  et Paul Gay
    </td>
</tr>
</table>

<br/>
<div id="top"></div>
<center>
    <a style="font-size: 20pt; font-weight: bold">Géometrie, caméra et Structure from Motion </a>
</center>
<br/>

##  Retrouvez les paramètres d'une transformation Affine

Dans cette première partie, nous verrons les différentes étapes permettant d'estimer une transformation entre deux images. 
* Détection de points de correspondances
* Construction du système d'équations
* Résolution par la méthode des moindres carrés.

In [ ]:
# some usefull imports
import matplotlib.pyplot as plt, matplotlib
%matplotlib inline  
import numpy as np
import cv2

### Obtenir l'image distordue

Complétez le code ci-dessous afin d'appliquer la transformation affine définie par les matrices A et B à l'image du babouin. Vous pouvez utiliser le même code que pour le cours précédent.

In [ ]:
infilename = "/home/paul/Documents/cours/eisti/slides/TI_01_image_convolutions/tmp/babouin.jpeg"
img = cv2.imread(infilename)
img = np.fliplr(img.reshape(-1,3)).reshape(img.shape)
distorted_image = np.zeros(img.shape)
A = np.array(((0.9, 0.1),(0.2,1)))
B = np.array((20, 20))
# votre code ici 

distorted_image = distorted_image.astype(np.uint8)
plt.imshow(distorted_image)
plt.axis('off')
matplotlib.rcParams['figure.figsize'] = [5, 5]


### Calcul des correspondances

Le code ci-dessous utilise les fonctions de matching de la librairie Opencv afin de trouver des points de correspondance entre l'image originale et l'image distordue. Il existe différents types de points d'intérêt dans la littérature. Nous utilisons ici les points ORB qui sont réputés pour la rapidité avec laquelle on les calcule et sont très utilisés en reconstruction 3D. 

Prenez le temps de faire varier les transformations A et B afin d'appliquer et de calculer les correspondances pour différentes paires d'images. Vous vous apercevrez que si l'image est très déformée, certaines correspondances, ne correspondrant pas aux mêmes parties du singe.

Que se passera t'il alors si nous calculons la transformation en prenant en compte ces points?


In [ ]:
# Initiate ORB detector
orb = cv2.ORB_create()
# find the keypoints and descriptors with ORB
# ... for the original image
kp1, des1 = orb.detectAndCompute(img,None)
# ... and for the transformed image
kp2, des2 = orb.detectAndCompute(distorted_image,None)
# create BFMatcher object
bf = cv2.BFMatcher(cv2.NORM_HAMMING, crossCheck=True)
# Match descriptors.
matches = bf.match(des1,des2)
# Sort them in the order of their distance.
matches = sorted(matches, key = lambda x:x.distance)
# Draw the first matches ie the ones for which the descriptors are similar.
number_of_matches = 20
img3 = cv2.drawMatches(img,kp1,distorted_image,kp2,matches[:number_of_matches],None,flags=cv2.DrawMatchesFlags_NOT_DRAW_SINGLE_POINTS)
matplotlib.rcParams['figure.figsize'] = [10, 10]
plt.imshow(img3)
plt.savefig("mismatch.png",  bbox_inches='tight')

#### Quelques mots sur les objets fournis par opencv. 

Le calcul de correspondances se basent en fait sur deux algorithmes : 
 * Un premier algorithme afin de détecter des candidats potentiels de points intéressants, qui seront facilement retrouvables dans l'image déformée. Typiquement, cela correspond aux zones contenant de la texture, et ne correspond pas aux zones de l'image uniformes.
 * Un deuxième algorithme servant à calculer un descripteur pour chaque point retenu. C'est ce descripteur qui est utilisé par la fonction `bf.match`


In [ ]:
num_correspondance = 0
print('Pour la correspondance : ', num_correspondance)
print("le point", matches[0].queryIdx, " de coordonnées",kp1[matches[0].queryIdx].pt, "de l'image originale")
print("... a une correspondance ... " )
print("avec le point", matches[0].trainIdx, "de coordonnées", kp2[matches[0].trainIdx].pt, "détecté sur l'image distordue")
print()
print("Nous pouvons vérifier qu'appliquer la transformation sur le premier point permet de retrouver un point proche du point détecté:")
projection = A.dot(np.array((kp1[matches[0].queryIdx].pt[1],kp1[matches[0].queryIdx].pt[0]))) + B
print('point détecté : ',kp2[matches[0].trainIdx].pt)
print('point projeté :', (projection[1], projection[0]))
print("C'est donc que cette paire sera utile pour estimer la transformation!")
print()

### Construction et résolution du système d'équations

Nous cherchons à présent les nouvelles matrices `Ap` et `Bp` telles que le point sur l'image orginale
```
kp1[matches[0].trainIdx]
```
soient proche de : 
```
Ap.dot(np.array((kp2[matches[0].queryIdx].pt[1],kp1[matches[0].queryIdx].pt[0]))) + Bp
```

Vous devez pour celà former le système d'équations, c'est à dire la matrice des observations U et p.

Afin que cela fonctionne, assurez vous que les correspondances que vous utilisez sont correctes et n'utilisez que les celles dont la distance est faible, par exemple, les 10 premières.

In [ ]:
n_match = 10 # number of matches used to compute
# votre code ici 


Utilisez maintenant la méthode de la pseudo inverse pour trouver les matrices Ap et Bp


In [ ]:
# votre code ici

### Vérification de la qualité de notre estimation

Utilisez la transformation que vous venez de calculer pour corriger l'image distordue et lui rendre sa forme originale. Vous pouvez ré-utiliser la cellule qui transforme l'image en changeant les matrices `A` et `B` par vos nouvelles matrices `Ap` et `Bp`.

In [ ]:
back_projected = np.zeros(distorted_image.shape)

# votre code ici

back_projected = back_projected.astype(np.uint8)
matplotlib.rcParams['figure.figsize'] = [10, 20]
fig=plt.figure()
fig.add_subplot(1,3,1)
imgplot = plt.imshow(img)
plt.axis('off')
fig.add_subplot(1,3,2)
imgplot = plt.imshow(distorted_image)
plt.axis('off')
fig.add_subplot(1,3,3)
imgplot = plt.imshow(back_projected)
plt.axis('off')
plt.show()


Faites variez le nombre de points d'intérêts utiliser lors de la transformation (variables `number_of_matches`). 

Observez que de mauvaises correspondances vont aboutir à une mauvaise transformation 